In [1]:
import random
import string
import time
import base64
from hashlib import sha256
from hmac import HMAC

import http.client
import requests
import pandas as pd 

APPID = "tubmafwrzhpgfiuf"
SECRET = "eotpcqbvhycdshwscqnytiwzbgonposs"


def generate_nonce(length=32):
    """Generate a random nonce."""
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))


def get_timestamp():
    """Get the current timestamp."""
    return int(time.time())


def build_sign_str(appid, nonce, timestamp):
    """Build the string to be signed."""
    return f'appid={appid}&nonce={nonce}&timestamp={timestamp}'


def calculate_signature(secret, message):
    """Calculate the HMAC SHA-256 signature."""
    return base64.urlsafe_b64encode(HMAC(secret.encode('utf-8'), message.encode('utf-8'), sha256).digest()).decode('utf-8')


def fetch_indicator_details(indicator_id):
    """Fetch the details for a specific indicator ID."""
    nonce = generate_nonce()
    timestamp = get_timestamp()
    sign_str = build_sign_str(APPID, nonce, timestamp)
    signature = calculate_signature(SECRET, sign_str)

    headers = {
        'nonce': nonce,
        'timestamp': str(timestamp),
        'appid': APPID,
        'signature': signature,
        'Accept': "*/*",
        'Accept-Encoding': "gzip, deflate, br",
        'User-Agent': "PostmanRuntime-ApipostRuntime/1.1.0",
        'Connection': "keep-alive",
    }

    url = f"https://etahub.hzinsights.com/v1/edb/data?EdbCode={indicator_id}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json().get('Data')
    else:
        print(
            f"Failed to fetch data for ID {indicator_id}, status code: {response.status_code}")
        return None

def fetch_indicator_name(indicator_id):
    """Fetch the name for a specific indicator ID."""
    nonce = generate_nonce()
    timestamp = get_timestamp()
    sign_str = build_sign_str(APPID, nonce, timestamp)
    signature = calculate_signature(SECRET, sign_str)

    headers = {
        'nonce': nonce,
        'timestamp': str(timestamp),
        'appid': APPID,
        'signature': signature,
        'Accept': "*/*",
        'Accept-Encoding': "gzip, deflate, br",
        'User-Agent': "PostmanRuntime-ApipostRuntime/1.1.0",
        'Connection': "keep-alive",
    }

    url = f"https://etahub.hzinsights.com/v1/edb/detail?EdbCode={indicator_id}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json().get('Data').get('EdbName')
    else:
        print(
            f"Failed to fetch data for ID {indicator_id}, status code: {response.status_code}")
        return None

def main():
    # List of indicator IDs you want to fetch
    indicator_ids = ["C2312015103",
                     'C2503281011443858',
                     'C2409261257520483',
                     'C2310167390',
                     'C2302050822',
                     'C2410041422486614',
                     'C2503281125238172',
                     'C2111039422',
                     'M0290205',
                     'WTICALMc1',
                     'C2410311101142339',
                     'C2504011050096928',
                     'C2503181311538813','C2505211632556267', 'C2505211650172587','LCOc1'
                     ]  # Add more IDs as needed

    # Dictionary to store DataFrames for each indicator
    data_frames = {}

    for indicator_id in indicator_ids:
        data = fetch_indicator_details(indicator_id)
        if data:
            # Create a DataFrame with DataTime as index
            df = pd.DataFrame(data)
            df['DataTime'] = pd.to_datetime(df['DataTime'])
            df.set_index('DataTime', inplace=True)
            df.sort_index(inplace=True)
            # Only keep the 'Value' column and rename it to the indicator ID
            df = df[['Value']].rename(columns={'Value': fetch_indicator_name(indicator_id)})
            data_frames[indicator_id] = df

    # Concatenate all DataFrames along the columns
    if data_frames:
        result_df = pd.concat(data_frames.values(), axis=1)
        print(result_df.info())
        result_df.to_excel("data_input_auto/山东柴油裂解差(多因子).xlsx")
        print("Data saved successfully as '山东柴油裂解差(多因子).xlsx'")



if __name__ == "__main__":
    main()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5830 entries, 2004-06-29 to 2025-10-31
Data columns (total 16 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   山东0#柴油净价-布伦特                                   3313 non-null   float64
 1   WTI连1合约价格（路透）20日环比                             2917 non-null   float64
 2   螺纹期货价格90日环比                                    3943 non-null   float64
 3   柴油：社会库存：中国（周）：大样本4周环差                          328 non-null    float64
 4   0#柴油价差（华东-山东）                                  3100 non-null   float64
 5   DR007 - 7天逆回购利率/15DMA90日环差                     2620 non-null   float64
 6   柴油：独立炼厂：产量：中国（周）超季节性/3年                        182 non-null    float64
 7   中国-美国10年期国债收益率差                                4345 non-null   float64
 8   USDCNH即期汇率                                     3504 non-null   float64
 9   WTI连1合约价格（路透）                     

Data saved successfully as '山东柴油裂解差(多因子).xlsx'
